## Métricas utilazadas para avaliação da recuperação dos chunks
- Tau
- Precision
- Recall

In [35]:
from flashrank import Ranker, RerankRequest
from scipy.stats import kendalltau
import pandas as pd
import time
import numpy as np
pd.set_option('display.max_rows', 100)  # Número de linhas que serão exibidas
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)  # Largura máxima para exibição
pd.set_option('display.max_colwidth', 500)  # Largura máxima para valores em células


In [20]:
query = "Quais são os principais desafios e aplicações da IA generativa na indústria atual?"
passages = [
   {
      "id": 1,
      "text": "A IA generativa tem ganhado destaque na indústria, especialmente com modelos como GPT e Diffusion, capazes de criar conteúdos originais e inovadores em diversas áreas. Sua aplicação vai desde geração de texto e imagens até novos designs e modelos de negócio.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 2,
      "text": "Na indústria atual, a IA generativa é utilizada em uma variedade de setores, incluindo chatbots para atendimento ao cliente, criação de imagens personalizadas para campanhas de marketing, design de produtos inovadores e produção de conteúdos audiovisuais, como roteiros e dublagens.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 3,
      "text": "Além das aplicações, a IA generativa enfrenta desafios importantes, como o viés nos modelos, a criação de deepfakes e a desinformação. Existe também a preocupação com o impacto nos empregos, especialmente em áreas de produção de conteúdo e design.",
      "meta": {"importância": "alta"}
   },
   {
      "id": 4,
      "text": "A IA generativa pode ter um impacto significativo na redução de custos de produção, acelerando processos criativos e aumentando a eficiência. No entanto, o alto custo de treinamento de modelos avançados e a necessidade de grande poder computacional ainda representam barreiras para muitas empresas.",
      "meta": {"importância": "média"}
   },
   {
      "id": 5,
      "text": "Embora a IA generativa traga avanços significativos em personalização de conteúdo, como recomendações de produtos e serviços em plataformas de streaming, seu uso mais generalizado em marketing digital e publicidade também levanta questões sobre privacidade e ética no uso de dados pessoais.",
      "meta": {"importância": "média"}
   },
   {
      "id": 6,
      "text": "O aquecimento global é uma das questões mais urgentes da atualidade, com impactos profundos nos ecossistemas e nas sociedades. As emissões de gases do efeito estufa continuam a aumentar, o que gera mudanças climáticas e eventos extremos cada vez mais frequentes.",
      "meta": {"importância": "baixa"}
   },
   {
      "id": 7,
      "text": "A história da arte é fascinante, cobrindo diversos períodos e movimentos que refletem as transformações culturais e sociais ao longo do tempo. Do Renascimento ao Modernismo, as expressões artísticas têm sido um reflexo dos avanços e desafios da humanidade.",
      "meta": {"importância": "baixa"}
   },
   {
      "id": 8,
      "text": "A exploração espacial continua a ser uma fronteira importante para a ciência e a tecnologia. Missões como as de Marte e a construção de estações espaciais internacionais têm ampliado o conhecimento humano sobre o universo e possibilitado novas tecnologias aplicáveis na Terra.",
      "meta": {"importância": "baixa"}
   }
]


In [6]:
ranker = Ranker(max_length=128)
rerankrequest = RerankRequest(query=query, passages=passages)

In [7]:
results = ranker.rerank(rerankrequest)
results

[{'id': 1,
  'text': 'A IA generativa tem ganhado destaque na indústria, especialmente com modelos como GPT e Diffusion, capazes de criar conteúdos originais e inovadores em diversas áreas. Sua aplicação vai desde geração de texto e imagens até novos designs e modelos de negócio.',
  'meta': {'importância': 'alta'},
  'score': 0.97679156},
 {'id': 3,
  'text': 'Além das aplicações, a IA generativa enfrenta desafios importantes, como o viés nos modelos, a criação de deepfakes e a desinformação. Existe também a preocupação com o impacto nos empregos, especialmente em áreas de produção de conteúdo e design.',
  'meta': {'importância': 'alta'},
  'score': 0.97494996},
 {'id': 5,
  'text': 'Embora a IA generativa traga avanços significativos em personalização de conteúdo, como recomendações de produtos e serviços em plataformas de streaming, seu uso mais generalizado em marketing digital e publicidade também levanta questões sobre privacidade e ética no uso de dados pessoais.',
  'meta': {'

In [ ]:
rerank_request = RerankRequest(query=query, passages=passages)
output = ranker.rerank(rerank_request)

# Extrai o ranking gerado pelo modelo (baseado nas pontuações)
predicted_ranking = [result["id"] for result in sorted(output, key=lambda x: x["score"], reverse=True)]
predicted_ranking

In [ ]:
predicted_ranking

[1, 3, 5, 2, 4, 8, 7, 6]

## Comparação de ordenação de modelos rerank entre diferente modelos

In [9]:
# Modelos a serem usados
rankers = [
    {"name": "Ranker(max_length=128)", "instance": Ranker(max_length=128)},
    {"name": "Ranker(model_name='ms-marco-MiniLM-L-12-v2')", "instance": Ranker(model_name="ms-marco-MiniLM-L-12-v2")},
    {"name": "Ranker(model_name='rank-T5-flan')", "instance": Ranker(model_name="rank-T5-flan")},
    {"name": "Ranker(model_name='ms-marco-MultiBERT-L-12')", "instance": Ranker(model_name="ms-marco-MultiBERT-L-12")},
]

# Função para calcular as métricas
def calculate_metrics(true_ranking, predicted_ranking):
    # Kendall's Tau
    tau, _ = kendalltau(true_ranking, predicted_ranking)
    
    return tau



In [23]:
true_ranking = [result["id"] for result in sorted(passages, key=lambda x: x["id"], reverse=False)]
true_ranking

[1, 2, 3, 4, 5, 6, 7, 8]

In [30]:
# Processamento para cada modelo
results = []
for ranker in rankers:
    execution_times = []  # Lista para armazenar os tempos de execução para cada iteração
    
    # Executa o código 5 vezes
    for _ in range(5):
        start_time = time.time()
        # Realiza o reranking com o modelo
        rerank_request = RerankRequest(query=query, passages=passages)
        output = ranker["instance"].rerank(rerank_request)
        end_time = time.time()
        
        # Calcula o tempo de execução para a iteração
        execution_time = end_time - start_time
        execution_times.append(execution_time)
    
    # Calcula o tempo médio e o tempo mediano
    avg_execution_time = np.mean(execution_times)
    median_execution_time = np.median(execution_times)
    
    # Extrai o ranking gerado pelo modelo (baseado nas pontuações)
    predicted_ranking = [result["id"] for result in sorted(output, key=lambda x: x["score"], reverse=True)]
    
    # Calcula as métricas
    tau = calculate_metrics(true_ranking, predicted_ranking)
    
    # Armazena os resultados, incluindo o tempo médio e o tempo mediano
    results.append({
        "Modelo": ranker["name"],
        "Métrica": tau,
        "true_ranking": true_ranking,
        "predicted_ranking": predicted_ranking,
        "Tempo Médio (segundos)": avg_execution_time,
        "Tempo Mediano (segundos)": median_execution_time
    })


In [31]:
df = pd.DataFrame(results)

# Ordenando pelo valor da métrica (do maior ao menor)
df_sorted = df.sort_values(by='Métrica', ascending=False)

# Exibindo o DataFrame ordenado
print(df_sorted)

                                         Modelo   Métrica  \
3  Ranker(model_name='ms-marco-MultiBERT-L-12')  0.785714   
1  Ranker(model_name='ms-marco-MiniLM-L-12-v2')  0.714286   
0                        Ranker(max_length=128)  0.571429   
2             Ranker(model_name='rank-T5-flan') -0.428571   

               true_ranking         predicted_ranking  Tempo Médio (segundos)  \
3  [1, 2, 3, 4, 5, 6, 7, 8]  [2, 1, 3, 5, 4, 6, 8, 7]                0.563338   
1  [1, 2, 3, 4, 5, 6, 7, 8]  [3, 1, 2, 5, 4, 7, 6, 8]                0.217655   
0  [1, 2, 3, 4, 5, 6, 7, 8]  [1, 3, 5, 2, 4, 8, 7, 6]                0.015556   
2  [1, 2, 3, 4, 5, 6, 7, 8]  [7, 8, 4, 5, 2, 3, 1, 6]                0.808404   

   Tempo Mediano (segundos)  
3                  0.574229  
1                  0.217379  
0                  0.012233  
2                  0.815950  


## Comparação de relevância(sem considerar ordenação) entre diferentes modelos de rerank

In [16]:
queries = [
    {
  "query": "Quais são os principais desafios e aplicações da IA generativa na indústria atual?",
  "passages": [
    {"id": 1, "text": "A IA generativa é utilizada em design de produtos e criação de roteiros audiovisuais.", "relevante": 1},
    {"id": 2, "text": "Desafios éticos incluem viés e impacto nos empregos causados pela IA generativa.", "relevante": 1},
    {"id": 3, "text": "A IA generativa oferece personalização em plataformas de streaming, com sugestões adaptadas ao gosto do usuário.", "relevante": 1},
    {"id": 4, "text": "A geração de deepfakes é um dos grandes desafios éticos associados à IA generativa.", "relevante": 1},
    {"id": 5, "text": "Uso de IA generativa em criação de protótipos rápidos para desenvolvimento de novos produtos.", "relevante": 1},
    {"id": 6, "text": "Aquecimento global é uma ameaça global causada pelas emissões de carbono.", "relevante": 0},
    {"id": 7, "text": "Movimentos artísticos como o Barroco influenciaram a história da arte.", "relevante": 0},
    {"id": 8, "text": "Exploração espacial tem avançado com missões tripuladas e construção de telescópios modernos.", "relevante": 0},
    {"id": 9, "text": "O uso de IA em medicina tem gerado grandes avanços, mas também apresenta questões éticas.", "relevante": 0},
    {"id": 10, "text": "Plataformas de e-commerce estão investindo em IA generativa para otimizar a personalização de anúncios.", "relevante": 0},
    {"id": 11, "text": "A IA generativa também é utilizada em melhorias na tradução automática e chatbots.", "relevante": 0},
    {"id": 12, "text": "O desenvolvimento de modelos de IA generativa exige grandes recursos computacionais e investimento.", "relevante": 0},
    {"id": 13, "text": "Impactos econômicos da IA generativa incluem redução de custos operacionais e aumento da produtividade.", "relevante": 0},
    {"id": 14, "text": "A inteligência artificial pode ser usada para prever padrões de comportamento de consumidores em diversos setores.", "relevante": 0},
    {"id": 15, "text": "O uso de IA generativa na criação de música é uma das fronteiras mais recentes da tecnologia.", "relevante": 0},
    {"id": 16, "text": "Desafios técnicos incluem controle das saídas e alinhamento com objetivos humanos.", "relevante": 0},
    {"id": 17, "text": "O aumento da produtividade em fábricas devido ao uso de IA é uma tendência crescente.", "relevante": 0},
    {"id": 18, "text": "Existem questões jurídicas em relação ao uso de IA generativa, como a autoria de conteúdos gerados.", "relevante": 0},
    {"id": 19, "text": "O impacto da IA generativa é significativo na criação de conteúdos audiovisuais para marketing.", "relevante": 1},
    {"id": 20, "text": "Robôs assistentes pessoais são uma inovação crescente em domicílios modernos.", "relevante": 0}
  ]
}
,
    {
    "query": "Quais são os impactos das mudanças climáticas no meio ambiente?",
    "passages": [
        {"id": 1, "text": "Mudanças climáticas resultam em eventos climáticos extremos e perda de biodiversidade.", "relevante": 1},
        {"id": 2, "text": "As emissões de carbono de indústrias aumentam o aquecimento global.", "relevante": 1},
        {"id": 3, "text": "O desmatamento contribui para o aumento das emissões de gases do efeito estufa.", "relevante": 1},
        {"id": 4, "text": "Mudanças climáticas podem afetar a agricultura, com secas prolongadas e redução das colheitas.", "relevante": 1},
        {"id": 5, "text": "O aumento do nível do mar ameaça várias cidades costeiras ao redor do mundo.", "relevante": 1},
        {"id": 6, "text": "A IA generativa pode criar deepfakes e conteúdos inovadores.", "relevante": 0},
        {"id": 7, "text": "A exploração espacial ajuda no desenvolvimento de novas tecnologias.", "relevante": 0},
        {"id": 8, "text": "Aquecimento global contribui para o derretimento das calotas polares.", "relevante": 0},
        {"id": 9, "text": "A preservação da natureza é essencial para combater as mudanças climáticas.", "relevante": 0},
        {"id": 10, "text": "Aquecimento global pode causar escassez de água em várias regiões do mundo.", "relevante": 0},
        {"id": 11, "text": "O aumento da temperatura global é uma das causas para o derretimento das geleiras.", "relevante": 0},
        {"id": 12, "text": "Mudanças no uso do solo têm um grande impacto no meio ambiente.", "relevante": 0},
        {"id": 13, "text": "Desastres naturais como terremotos não têm relação direta com as mudanças climáticas.", "relevante": 0},
        {"id": 14, "text": "O desenvolvimento de novas energias renováveis pode ajudar a mitigar os impactos das mudanças climáticas.", "relevante": 0},
        {"id": 15, "text": "O aumento da poluição também afeta os ecossistemas e contribui para o aquecimento global.", "relevante": 0},
        {"id": 16, "text": "A evolução da medicina tem impactado positivamente a qualidade de vida das pessoas.", "relevante": 0},
        {"id": 17, "text": "O impacto do turismo sobre o meio ambiente é um tema de grande discussão atualmente.", "relevante": 0},
        {"id": 18, "text": "A tecnologia tem sido fundamental para o desenvolvimento de novas formas de combate às mudanças climáticas.", "relevante": 0},
        {"id": 19, "text": "A vida marinha está sendo seriamente afetada pela acidificação dos oceanos devido às mudanças climáticas.", "relevante": 0},
        {"id": 20, "text": "Aquecimento global é causado pela queima de combustíveis fósseis e desmatamento.", "relevante": 0}
    ]
}

,
    {
    "query": "Quais são os avanços recentes na exploração espacial?",
    "passages": [
        {"id": 1, "text": "Missões para Marte têm aumentado o conhecimento sobre o planeta.", "relevante": 1},
        {"id": 2, "text": "Novas tecnologias aplicadas na construção de estações espaciais.", "relevante": 1},
        {"id": 3, "text": "Exploração espacial pode ajudar na mineração de asteroides.", "relevante": 1},
        {"id": 4, "text": "Desenvolvimento de novas naves espaciais para missões tripuladas à Lua e Marte.", "relevante": 1},
        {"id": 5, "text": "Tecnologias de propulsão avançada estão sendo desenvolvidas para missões espaciais de longa duração.", "relevante": 1},
        {"id": 6, "text": "IA generativa é usada em dublagem e criação de textos.", "relevante": 0},
        {"id": 7, "text": "O aumento do turismo espacial está gerando novas oportunidades econômicas.", "relevante": 0},
        {"id": 8, "text": "Aquecimento global afeta os ecossistemas marinhos e terrestres.", "relevante": 0},
        {"id": 9, "text": "O uso de satélites para monitoramento climático tem avançado significativamente.", "relevante": 0},
        {"id": 10, "text": "Exploração de novos planetas fora do sistema solar ainda está em fase preliminar.", "relevante": 0},
        {"id": 11, "text": "A pesquisa sobre buracos negros e fenômenos astrofísicos está em expansão.", "relevante": 0},
        {"id": 12, "text": "Os avanços na medicina têm contribuído para missões espaciais mais longas e seguras.", "relevante": 0},
        {"id": 13, "text": "Exploração de asteroides em busca de recursos minerais é um campo emergente.", "relevante": 0},
        {"id": 14, "text": "A colaboração internacional em missões espaciais está se expandindo a cada ano.", "relevante": 0},
        {"id": 15, "text": "A nova geração de telescópios espaciais oferece imagens mais detalhadas do universo.", "relevante": 0},
        {"id": 16, "text": "O avanço da computação quântica tem sido um fator importante para a exploração espacial.", "relevante": 0},
        {"id": 17, "text": "O impacto das mudanças climáticas no espaço ainda é um campo pouco explorado.", "relevante": 0},
        {"id": 18, "text": "A tecnologia 5G permitirá avanços na comunicação durante missões espaciais.", "relevante": 0},
        {"id": 19, "text": "O uso de drones para exploração de planetas distantes está sendo estudado.", "relevante": 0},
        {"id": 20, "text": "A pesquisa sobre vida extraterrestre é um dos objetivos das missões espaciais.", "relevante": 0}
    ]
}
,
    {
    "query": "Como a IA está transformando a área da saúde?",
    "passages": [
        {"id": 1, "text": "IA é usada para diagnósticos médicos e simulações de medicamentos.", "relevante": 1},
        {"id": 2, "text": "Modelos de IA ajudam na personalização de tratamentos médicos.", "relevante": 1},
        {"id": 3, "text": "IA auxilia no monitoramento remoto de pacientes.", "relevante": 1},
        {"id": 4, "text": "IA é aplicada no desenvolvimento de novos medicamentos através de simulações computacionais.", "relevante": 1},
        {"id": 5, "text": "Sistemas de IA são usados em hospitais para gerenciar dados de pacientes e otimizar tratamentos.", "relevante": 1},
        {"id": 6, "text": "Deepfakes gerados por IA podem ser uma ameaça à privacidade.", "relevante": 0},
        {"id": 7, "text": "Aquecimento global resulta na mudança de padrões climáticos ao redor do mundo.", "relevante": 0},
        {"id": 8, "text": "O uso de IA na educação está revolucionando o ensino personalizado.", "relevante": 0},
        {"id": 9, "text": "Exploração de Marte está levando a avanços na tecnologia de IA para o ambiente espacial.", "relevante": 0},
        {"id": 10, "text": "Avanços em biotecnologia têm impactado diretamente a medicina moderna.", "relevante": 0},
        {"id": 11, "text": "A IA pode ajudar na detecção precoce de doenças, aumentando as taxas de cura.", "relevante": 0},
        {"id": 12, "text": "A utilização de IA para triagem de pacientes está sendo testada em vários hospitais.", "relevante": 0},
        {"id": 13, "text": "IA está transformando a agricultura ao melhorar a produtividade e a sustentabilidade.", "relevante": 0},
        {"id": 14, "text": "Tecnologias de IA permitem avanços na indústria automobilística, como carros autônomos.", "relevante": 0},
        {"id": 15, "text": "O uso de IA para otimizar as operações logísticas tem sido cada vez mais comum.", "relevante": 0},
        {"id": 16, "text": "Modelos preditivos baseados em IA podem ser aplicados em diversas áreas, como finanças e saúde.", "relevante": 0},
        {"id": 17, "text": "O impacto da IA no mercado de trabalho está sendo debatido globalmente.", "relevante": 0},
        {"id": 18, "text": "Os benefícios da IA para a indústria de entretenimento incluem a criação de conteúdos personalizados.", "relevante": 0},
        {"id": 19, "text": "A IA está sendo explorada para melhorar a segurança no setor financeiro.", "relevante": 0},
        {"id": 20, "text": "A IA tem sido fundamental para a evolução dos sistemas de inteligência policial.", "relevante": 0}
    ]
}
,
    {
    "query": "Qual é o impacto do uso de IA na indústria de entretenimento?",
    "passages": [
        {"id": 1, "text": "IA cria conteúdos como roteiros e efeitos visuais em filmes.", "relevante": 1},
        {"id": 2, "text": "Personalização de playlists e recomendações em plataformas de streaming.", "relevante": 1},
        {"id": 3, "text": "IA ajuda na análise de dados para criar experiências imersivas.", "relevante": 1},
        {"id": 4, "text": "O uso de IA em jogos permite a criação de personagens e ambientes mais realistas.", "relevante": 1},
        {"id": 5, "text": "Plataformas de streaming utilizam IA para recomendar filmes e músicas com base no gosto do usuário.", "relevante": 1},
        {"id": 6, "text": "O aquecimento global está causando derretimento de calotas polares.", "relevante": 0},
        {"id": 7, "text": "A IA está transformando a indústria automotiva com carros autônomos.", "relevante": 0},
        {"id": 8, "text": "Avanços tecnológicos em medicina estão impactando diretamente os tratamentos de saúde.", "relevante": 0},
        {"id": 9, "text": "A IA é usada para personalizar anúncios e estratégias de marketing digital.", "relevante": 0},
        {"id": 10, "text": "Exploração espacial avança com novas missões para outros planetas.", "relevante": 0},
        {"id": 11, "text": "A utilização de IA em carros autônomos promete melhorar a segurança no trânsito.", "relevante": 0},
        {"id": 12, "text": "A IA pode ser usada para otimizar processos logísticos em grandes empresas.", "relevante": 0},
        {"id": 13, "text": "Tecnologias de IA são cada vez mais usadas para o reconhecimento de padrões em dados financeiros.", "relevante": 0},
        {"id": 14, "text": "A IA tem sido aplicada no setor de segurança, ajudando na vigilância e detecção de crimes.", "relevante": 0},
        {"id": 15, "text": "A IA também é usada para melhorar a eficiência na produção de energia e no setor industrial.", "relevante": 0},
        {"id": 16, "text": "Modelos de IA estão sendo empregados em vários campos da educação para personalizar o aprendizado.", "relevante": 0},
        {"id": 17, "text": "A IA tem um papel crescente em plataformas de e-commerce, ajudando a criar experiências personalizadas para os consumidores.", "relevante": 0},
        {"id": 18, "text": "Sistemas de IA são utilizados para otimizar a gestão de resíduos e melhorar a sustentabilidade ambiental.", "relevante": 0},
        {"id": 19, "text": "O impacto da IA no mercado de trabalho é tema de muitos debates atualmente.", "relevante": 0},
        {"id": 20, "text": "A IA pode ser usada para criar simulações e experiências de realidade aumentada em jogos e filmes.", "relevante": 0}
    ]
}

]


In [20]:
def precision_at_k(results, k):
    """
    Calcula a Precision@k de um modelo de reranking.
    
    :param results: lista de resultados do modelo, onde cada resultado é um dicionário com as chaves 'id', 'text', 'relevante', e 'score'.
    :param k: o número de documentos a considerar para calcular Precision@k.
    :return: a Precision@k, que é a fração de documentos relevantes nos k primeiros resultados.
    """
    # Ordena os resultados pela pontuação (score), do maior para o menor
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    # Seleciona os k primeiros resultados
    top_k_results = sorted_results[:k]
    
    # Conta quantos dos k primeiros são relevantes
    relevant_count = sum([1 for result in top_k_results if result['relevante'] == 1])
    
    # Calcula Precision@k
    precision_k = relevant_count / k
    return precision_k

In [23]:
def recall_at_k(results, k):
    """
    Calcula o Recall@k de um modelo de reranking.
    
    :param results: lista de resultados do modelo, onde cada resultado é um dicionário com as chaves 'id', 'text', 'relevante', e 'score'.
    :param relevant_docs: lista de ids dos documentos relevantes no conjunto completo.
    :param k: o número de documentos a considerar para calcular Recall@k.
    :return: o Recall@k, que é a fração de documentos relevantes recuperados nos k primeiros resultados.
    """
    # Ordena os resultados pela pontuação (score), do maior para o menor
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    # Seleciona os k primeiros resultados
    top_k_results = sorted_results[:k]
    
    # Conta quantos dos k primeiros são relevantes
    retrieved_relevant_count = sum([1 for result in top_k_results if result['relevante'] == 1])
    
    # Total de documentos relevantes no conjunto
    total_relevant_docs = len([result for result in results if result['relevante'] == 1])
    
    # Calcula Recall@k
    recall_k = retrieved_relevant_count / total_relevant_docs if total_relevant_docs > 0 else 0
    return recall_k


In [25]:
k = 5
precision = precision_at_k(results, k)
print(f"Precision@{k}: {precision}")

Precision@5: 0.4


In [24]:
k = 5
recall = recall_at_k(results, k)
print(f"Recall@{k}: {recall}")

Recall@5: 0.3333333333333333


In [26]:
rankers = [
    {"name": "Ranker(max_length=128)", "instance": Ranker(max_length=128)},
    {"name": "Ranker(model_name='ms-marco-MiniLM-L-12-v2')", "instance": Ranker(model_name="ms-marco-MiniLM-L-12-v2")},
    {"name": "Ranker(model_name='rank-T5-flan')", "instance": Ranker(model_name="rank-T5-flan")},
    {"name": "Ranker(model_name='ms-marco-MultiBERT-L-12')", "instance": Ranker(model_name="ms-marco-MultiBERT-L-12")},
]

In [ ]:
# Inicializa a lista para armazenar os resultados finais
overall_results = []

# Define os valores de k para calcular as métricas
k_values = [3, 5, 8, 10]

# Itera sobre os modelos
for ranker in rankers:
    model_results = []
    
    # Itera sobre as queries
    for query_data in queries:
        query_text = query_data["query"]
        passages = query_data["passages"]
        
        start_time = time.time()
        
        # Realiza o reranking para o modelo atual
        rerank_request = RerankRequest(query=query_text, passages=passages)
        output = ranker["instance"].rerank(rerank_request)
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        # Itera sobre os valores de k
        for k in k_values:
            precision = precision_at_k(output, k)
            recall = recall_at_k(output, k)
            
            # Armazena os resultados desta query
            model_results.append({
                "Modelo": ranker["name"],
                "Query": query_text,
                "k": k,
                "Tempo de Execução (segundos)": execution_time,
                "Precision@k": precision,
                "Recall@k": recall
            })
    
    # Após calcular para todas as queries, calcula as médias por k
    for k in k_values:
        precision_avg = np.mean([res["Precision@k"] for res in model_results if res["k"] == k])
        recall_avg = np.mean([res["Recall@k"] for res in model_results if res["k"] == k])
        
        overall_results.append({
            "Modelo": ranker["name"],
            "k": k,
            "Tempo Médio de Execução (segundos)": np.mean([res["Tempo de Execução (segundos)"] for res in model_results if res["k"] == k]),
            "Precision Média": precision_avg,
            "Recall Média": recall_avg
        })

# Converte os resultados para um DataFrame do Pandas
df_overall_results = pd.DataFrame(overall_results)

# Exibe os resultados
df_sorted = df_overall_results.sort_values(by=["Modelo", "k"], ascending=[True, True])



                                          Modelo  ...  Recall Média
0                         Ranker(max_length=128)  ...      0.153333
1                         Ranker(max_length=128)  ...      0.266667
2                         Ranker(max_length=128)  ...      0.373333
3                         Ranker(max_length=128)  ...      0.406667
4   Ranker(model_name='ms-marco-MiniLM-L-12-v2')  ...      0.146667
5   Ranker(model_name='ms-marco-MiniLM-L-12-v2')  ...      0.220000
6   Ranker(model_name='ms-marco-MiniLM-L-12-v2')  ...      0.300000
7   Ranker(model_name='ms-marco-MiniLM-L-12-v2')  ...      0.373333
12  Ranker(model_name='ms-marco-MultiBERT-L-12')  ...      0.186667
13  Ranker(model_name='ms-marco-MultiBERT-L-12')  ...      0.266667
14  Ranker(model_name='ms-marco-MultiBERT-L-12')  ...      0.380000
15  Ranker(model_name='ms-marco-MultiBERT-L-12')  ...      0.453333
8              Ranker(model_name='rank-T5-flan')  ...      0.080000
9              Ranker(model_name='rank-T5-flan')

In [39]:
print(df_sorted.sort_values(by="Recall Média", ascending=False))

                                          Modelo   k  Tempo Médio de Execução (segundos)  Precision Média  Recall Média
11             Ranker(model_name='rank-T5-flan')  10                            0.987416         0.260000      0.500000
15  Ranker(model_name='ms-marco-MultiBERT-L-12')  10                            0.767765         0.240000      0.453333
3                         Ranker(max_length=128)  10                            0.013757         0.220000      0.406667
14  Ranker(model_name='ms-marco-MultiBERT-L-12')   8                            0.767765         0.250000      0.380000
2                         Ranker(max_length=128)   8                            0.013757         0.250000      0.373333
7   Ranker(model_name='ms-marco-MiniLM-L-12-v2')  10                            0.268538         0.200000      0.373333
10             Ranker(model_name='rank-T5-flan')   8                            0.987416         0.225000      0.346667
6   Ranker(model_name='ms-marco-MiniLM-L